In [ ]:
import easyocr
import pandas as pd
from PIL import Image
import csv
from wordsegment import segment, load

cluster = ''

In [ ]:
load()

reader = easyocr.Reader(['en'])
fetched = pd.read_csv(f'{cluster}.csv')
ocrd = open(f'{cluster}-top-ocr.csv', "w", newline='', encoding="utf-8")
writer = csv.writer(ocrd)
writer.writerow(['post_name', 'text', 'caption'])

imgs = [*fetched['img_path']]

In [ ]:
def extract_image_from_gif(n):
    with Image.open(f"{cluster}/{n}.gif") as im:
        im.seek(0)
        im.save(f"{cluster}/{n}.png")

def easy_ocr(path):
    result = reader.readtext(path, detail=0, paragraph=True, y_ths = -0.05, text_threshold=0.85)
    return result

def word_segment(sentences):
    return "\n".join([" ".join(segment(sentence)) for sentence in sentences])

# def clean_sentence(sentences):
#     nl = ['mematic', 'imgflip']
#     def check(sentence, nl):
#         return not any([all([w in na.split() for w in sentence]) for na in nl])
#     return [sentence for sentence in sentences if check(sentence, nl)]

def read_meme(path):
    [n, t] = path.split(".")
    if t == "gif":
        extract_image_from_gif(n)
    # return word_segment(easy_ocr(f"{cluster}/{n}.{t if t != 'gif' else 'png'}"))
    text = word_segment(easy_ocr(f"{cluster}/{n}.{t if t != 'gif' else 'png'}"))
    # print(text)
    return path, text

In [ ]:
for i, img in enumerate((imgs)):
  if(img.split(".")[-1] == "gifv"):
      continue
  res = read_meme(img)
  writer.writerow([res[0].split(".")[0], res[1], fetched.loc[fetched['img_path'] == res[0]]['post_caption'].tolist()[0]])
  print(f"{i}: writing", res[0])